# SITS (Spatial ITS) Analysis

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [2]:
''' Read Data - Choose ONE of the following datasets '''

# Travel Distance per trip: LAD
# file = 'mobility indicators/daily_alltrips_duration_distance_GLAresidence_homearea_pertrip/mean_distance_lad_merge_result.csv'
# area_key = 'LAD22CD'
# use_col = 'GLA'

# Travel Distance per trip: MSOA
file = 'mobility indicators/daily_alltrips_duration_distance_GLAresidence_homearea_pertrip/mean_distance_msoa_merge_result.csv'
area_key = 'MSOA21CD'
use_col = 'GLA'

# Casual Activity Count: LAD
# file = 'mobility indicators/daily_count_casual_activities/merge_result_lad.csv'
# area_key = 'LAD22CD'
# use_col = 'activity_count'

# Casual Activity Count: MSOA
# file = 'mobility indicators/daily_count_casual_activities/merge_result_msoa.csv'
# area_key = 'MSOA21CD'
# use_col = 'activity_count'

# Commuting Duration per capita: LAD
# file = 'mobility indicators/daily_commuting_duration_distance/mean_duration_distance_lad_merge_result.csv'
# area_key = 'LAD22CD'
# use_col = 'duration_minutes'

# Commuting Duration per capita: MSOA
# file = 'mobility indicators/daily_commuting_duration_distance/mean_duration_distance_msoa_merge_result.csv'
# area_key = 'MSOA21CD'
# use_col = 'duration_minutes'

In [3]:
''' Read Data '''

data = pd.read_csv(file)
data.rename(columns={use_col: 'count'}, inplace=True)
folder_name = file.split('/')[1]

# Convert 'day' to datetime
data['day'] = pd.to_datetime(data['day'])

# select data before 2021-12-12, remove Christmas
data = data[data['day'] < ('2021-12-12')]
data = data.groupby(['day', area_key]).agg({'count': 'mean'}).reset_index()

# Create a binary indicator for the intervention (1 if on or after Oct 25, 2021, else 0)
intervention_date = pd.to_datetime('2021-10-25')
data['intervention'] = (data['day'] >= intervention_date).astype(int)

# Create time variables (number of hours since the start of the dataset)
data['time1'] = (data['day'] - data['day'].min()).dt.total_seconds() / (60 * 60 * 24)
data['time2'] = (data['day'] - intervention_date).dt.total_seconds() / (60 * 60 * 24)
data.loc[data['time2'] < 0, 'time2'] = 0

# Create Day of Week (dow) categorical variable
data['dow'] = data['day'].dt.dayofweek
day_of_week_dummies = pd.get_dummies(data['dow'], prefix='dow', drop_first=True).astype(int)
data = pd.concat([data, day_of_week_dummies], axis=1)
data.drop(columns=['dow'], inplace=True)

data

,day,MSOA21CD,count,intervention,time1,time2,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
0,2021-10-01,E02000001,4217.592031,0,0.0,0.0,0,0,0,1,0,0
1,2021-10-01,E02000002,3052.038883,0,0.0,0.0,0,0,0,1,0,0
2,2021-10-01,E02000003,4271.400031,0,0.0,0.0,0,0,0,1,0,0
3,2021-10-01,E02000004,3316.678252,0,0.0,0.0,0,0,0,1,0,0
4,2021-10-01,E02000005,2820.681639,0,0.0,0.0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
72139,2021-12-11,E02007111,3825.484783,1,71.0,47.0,0,0,0,0,1,0
72140,2021-12-11,E02007112,3952.935414,1,71.0,47.0,0,0,0,0,1,0
72141,2021-12-11,E02007113,4513.575395,1,71.0,47.0,0,0,0,0,1,0
72142,2021-12-11,E02007114,4052.152848,1,71.0,47.0,0,0,0,0,1,0


In [4]:
data['count'].mean()

3595.012119886288

In [5]:
# remove outliers: City of London

''' LAD '''
# data = data[data['LAD22CD'] != 'E09000001']
# data

''' MSOA '''
data = data[data['MSOA21CD'] != 'E02000001']
data

,day,MSOA21CD,count,intervention,time1,time2,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
1,2021-10-01,E02000002,3052.038883,0,0.0,0.0,0,0,0,1,0,0
2,2021-10-01,E02000003,4271.400031,0,0.0,0.0,0,0,0,1,0,0
3,2021-10-01,E02000004,3316.678252,0,0.0,0.0,0,0,0,1,0,0
4,2021-10-01,E02000005,2820.681639,0,0.0,0.0,0,0,0,1,0,0
5,2021-10-01,E02000007,2999.628000,0,0.0,0.0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
72139,2021-12-11,E02007111,3825.484783,1,71.0,47.0,0,0,0,0,1,0
72140,2021-12-11,E02007112,3952.935414,1,71.0,47.0,0,0,0,0,1,0
72141,2021-12-11,E02007113,4513.575395,1,71.0,47.0,0,0,0,0,1,0
72142,2021-12-11,E02007114,4052.152848,1,71.0,47.0,0,0,0,0,1,0


In [6]:
# Read Census Data

''' LAD level '''
"""
# rden: residence density, TS0006
rden = pd.read_csv('Census Data/census2021-ts006-ltla.csv').iloc[:, 2:]
rden.columns = ['LAD22CD', 'Rden']
data = data.merge(rden, on='LAD22CD', how='left')

# age: average age, TS0007
# age = pd.read_csv('Census Data/census2021-ts007-ltla.csv')

# Sfem: Share of femal, TS0008
sfem = pd.read_csv('Census Data/census2021-ts008-ltla.csv').iloc[:, 2:]
sfem['Sfem'] = sfem['Sex: Female; measures: Value'] / sfem['Sex: All persons; measures: Value']
sfem = sfem[['geography code', 'Sfem']]
sfem.columns = ['LAD22CD', 'Sfem']
data = data.merge(sfem, on='LAD22CD', how='left')

# Hedu: Higher education, TS0067
hedu = pd.read_csv('Census Data/census2021-ts067-ltla.csv')
hedu['Hedu'] = hedu['Highest level of qualification: Level 4 qualifications and above'] / hedu['Highest level of qualification: Total: All usual residents aged 16 years and over']
hedu = hedu[['geography code', 'Hedu']]
hedu.columns = ['LAD22CD', 'Hedu']
data = data.merge(hedu, on='LAD22CD', how='left')

# Scar: Share of car ownership, TS0045
scar = pd.read_csv('Census Data/census2021-ts045-ltla.csv').iloc[:, 2:]
scar['Scar'] = (scar['Number of cars or vans: Total: All households'] - scar['Number of cars or vans: No cars or vans in household']) / scar['Number of cars or vans: Total: All households']
scar = scar[['geography code', 'Scar']]
scar.columns = ['LAD22CD', 'Scar']
data = data.merge(scar, on='LAD22CD', how='left')

# DistWork: Distance to work, TS0058
# distwork = pd.read_csv('Census Data/census2021-ts058-ltla.csv').iloc[:, 2:]

# CarWork: Share of car to work, TS0061
carwork = pd.read_csv('Census Data/census2021-ts061-ltla.csv').iloc[:, 2:]
carwork['CarWork'] = carwork['Method of travel to workplace: Driving a car or van'] / carwork['Method of travel to workplace: Total: All usual residents aged 16 years and over in employment the week before the census']
carwork = carwork[['geography code', 'CarWork']]
carwork.columns = ['LAD22CD', 'CarWork']
data = data.merge(carwork, on='LAD22CD', how='left')

# Swfh: Share of working from home, TS0061
swfh = pd.read_csv('Census Data/census2021-ts061-ltla.csv').iloc[:, 2:]
swfh['Swfh'] = swfh['Method of travel to workplace: Work mainly at or from home'] / swfh['Method of travel to workplace: Total: All usual residents aged 16 years and over in employment the week before the census']
swfh = swfh[['geography code', 'Swfh']]
swfh.columns = ['LAD22CD', 'Swfh']
data = data.merge(swfh, on='LAD22CD', how='left')

# Income: GDHI
gdhi = pd.read_csv('Census Data/income_GDHI2021_LondonLAD.csv', usecols=['mnemonic', 'GDHI per head index (UK = 100)'])
gdhi.columns = ['LAD22CD', 'Income']
data = data.merge(gdhi, on='LAD22CD', how='left')

# PTAL: Public Transport Accessibility Level
ptal = pd.read_csv('PTAL/LAD2022 AvPTAI2015.csv')
ptal.columns = ['LAD22CD', 'PTAL']
data = data.merge(ptal, on='LAD22CD', how='left')

social_vars = ['Rden', 'Sfem', 'Hedu', 'Scar', 'CarWork', 'Swfh', 'Income', 'PTAL']
"""


''' MSOA level '''
# """
# rden: residence density, TS0006
rden = pd.read_csv('Census Data/census2021-ts006-msoa.csv').iloc[:, 2:]
rden.columns = ['MSOA21CD', 'Rden']
data = data.merge(rden, on='MSOA21CD', how='left')

# age: average age, TS0007
# age = pd.read_csv('/Users/yk/Documents/Code/ChargingZone/Census Data/census2021-ts007-msoa.csv')

# Sfem: Share of femal, TS0008
sfem = pd.read_csv('Census Data/census2021-ts008-msoa.csv').iloc[:, 2:]
sfem['Sfem'] = sfem['Sex: Female; measures: Value'] / sfem['Sex: All persons; measures: Value']
sfem = sfem[['geography code', 'Sfem']]
sfem.columns = ['MSOA21CD', 'Sfem']
data = data.merge(sfem, on='MSOA21CD', how='left')

# Hedu: Higher education, TS0067
hedu = pd.read_csv('Census Data/census2021-ts067-msoa.csv')
hedu['Hedu'] = hedu['Highest level of qualification: Level 4 qualifications and above'] / hedu['Highest level of qualification: Total: All usual residents aged 16 years and over']
hedu = hedu[['geography code', 'Hedu']]
hedu.columns = ['MSOA21CD', 'Hedu']
data = data.merge(hedu, on='MSOA21CD', how='left')

# Scar: Share of car ownership, TS0045
scar = pd.read_csv('Census Data/census2021-ts045-msoa.csv').iloc[:, 2:]
scar['Scar'] = (scar['Number of cars or vans: Total: All households'] - scar['Number of cars or vans: No cars or vans in household']) / scar['Number of cars or vans: Total: All households']
scar = scar[['geography code', 'Scar']]
scar.columns = ['MSOA21CD', 'Scar']
data = data.merge(scar, on='MSOA21CD', how='left')

# DistWork: Distance to work, TS0058
# distwork = pd.read_csv('Census Data/census2021-ts058-msoa.csv').iloc[:, 2:]

# CarWork: Share of car to work, TS0061
carwork = pd.read_csv('Census Data/census2021-ts061-msoa.csv').iloc[:, 2:]
carwork['CarWork'] = carwork['Method of travel to workplace: Driving a car or van'] / carwork['Method of travel to workplace: Total: All usual residents aged 16 years and over in employment the week before the census']
carwork = carwork[['geography code', 'CarWork']]
carwork.columns = ['MSOA21CD', 'CarWork']
data = data.merge(carwork, on='MSOA21CD', how='left')

# Swfh: Share of working from home, TS0061
swfh = pd.read_csv('Census Data/census2021-ts061-msoa.csv').iloc[:, 2:]
swfh['Swfh'] = swfh['Method of travel to workplace: Work mainly at or from home'] / swfh['Method of travel to workplace: Total: All usual residents aged 16 years and over in employment the week before the census']
swfh = swfh[['geography code', 'Swfh']]
swfh.columns = ['MSOA21CD', 'Swfh']
data = data.merge(swfh, on='MSOA21CD', how='left')

# PTAL: Public Transport Accessibility Level
ptal = pd.read_csv('PTAL/MSOA2021 AvPTAI2015.csv')
ptal.columns = ['MSOA21CD', 'PTAI']
data = data.merge(ptal, on='MSOA21CD', how='left')

social_vars = ['Rden', 'Sfem', 'Hedu', 'Scar', 'CarWork', 'Swfh', 'PTAI']
# """

''' centering '''
for var in social_vars:
    data[f'{var}_centered'] = data[var] - data[var].mean()

# fillna
data.fillna(0, inplace=True)

data

,day,MSOA21CD,count,intervention,time1,time2,dow_1,dow_2,dow_3,dow_4,...,CarWork,Swfh,PTAI,Rden_centered,Sfem_centered,Hedu_centered,Scar_centered,CarWork_centered,Swfh_centered,PTAI_centered
0,2021-10-01,E02000002,3052.038883,0,0.0,0.0,0,0,0,1,...,0.390364,0.209389,3.731210,-5060.063237,0.011241,-0.158924,0.090805,0.179302,-0.207410,-9.819832
1,2021-10-01,E02000003,4271.400031,0,0.0,0.0,0,0,0,1,...,0.320351,0.255059,7.723008,-3501.363237,-0.010048,-0.127878,0.115125,0.109289,-0.161740,-5.828034
2,2021-10-01,E02000004,3316.678252,0,0.0,0.0,0,0,0,1,...,0.391779,0.250803,4.164166,-6227.463237,-0.004362,-0.160598,0.173672,0.180717,-0.165996,-9.386876
3,2021-10-01,E02000005,2820.681639,0,0.0,0.0,0,0,0,1,...,0.355821,0.220485,6.134339,436.436763,-0.002964,-0.130190,0.129228,0.144758,-0.196314,-7.416703
4,2021-10-01,E02000007,2999.628000,0,0.0,0.0,0,0,0,1,...,0.344803,0.186798,5.266994,-3036.663237,0.011161,-0.150993,0.035123,0.133741,-0.230001,-8.284047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72067,2021-12-11,E02007111,3825.484783,1,71.0,47.0,0,0,0,0,...,0.034812,0.629448,42.214395,312.236763,-0.020970,0.209927,-0.401807,-0.176251,0.212649,28.663354
72068,2021-12-11,E02007112,3952.935414,1,71.0,47.0,0,0,0,0,...,0.049000,0.681750,12.478780,6226.936763,-0.044609,0.236516,-0.360011,-0.162062,0.264951,-1.072262
72069,2021-12-11,E02007113,4513.575395,1,71.0,47.0,0,0,0,0,...,0.065438,0.596544,15.832084,1811.836763,-0.032327,0.157646,-0.295962,-0.145625,0.179745,2.281042
72070,2021-12-11,E02007114,4052.152848,1,71.0,47.0,0,0,0,0,...,0.072718,0.568014,13.968328,8507.736763,-0.019406,0.111075,-0.274435,-0.138344,0.151215,0.417286


In [7]:
# Read time series data: fuel price

''' Fuel weekly data '''
weekly_data = pd.read_csv('/Users/yk/Documents/Code/ChargingZone/fuel price/weekly_fuel.csv')
# 将日期转换为日期格式
weekly_data['Date'] = pd.to_datetime(weekly_data['Date'], format='%m/%d/%y')
# 创建每日数据框
daily_data = pd.DataFrame({
    'Date': pd.date_range(start='2021-09-27', end='2021-12-11', freq='D'),
    'fuel_price': 0  # 示例变量
})
# 将每周数据扩展为每日数据
daily_data = daily_data.merge(
    weekly_data.rename(columns={'Date': 'Weekly_Date'}),
    how='left',
    left_on='Date',
    right_on='Weekly_Date'
)
# 向前填充每周价格值
daily_data['fuel_price'] = daily_data['ULSP'].ffill()
# 删除多余的列
daily_data = daily_data.drop(columns=['Weekly_Date', 'ULSP'])
# 中心化：政策日变为0
# daily_data['fuel_price'] = daily_data['fuel_price'] - daily_data[daily_data['Date']=='2021-10-25']['fuel_price']

data = data.merge(daily_data, left_on='day', right_on='Date', how='left')
data.drop(columns=['Date'], inplace=True)
data

,day,MSOA21CD,count,intervention,time1,time2,dow_1,dow_2,dow_3,dow_4,...,Swfh,PTAI,Rden_centered,Sfem_centered,Hedu_centered,Scar_centered,CarWork_centered,Swfh_centered,PTAI_centered,fuel_price
0,2021-10-01,E02000002,3052.038883,0,0.0,0.0,0,0,0,1,...,0.209389,3.731210,-5060.063237,0.011241,-0.158924,0.090805,0.179302,-0.207410,-9.819832,135.19
1,2021-10-01,E02000003,4271.400031,0,0.0,0.0,0,0,0,1,...,0.255059,7.723008,-3501.363237,-0.010048,-0.127878,0.115125,0.109289,-0.161740,-5.828034,135.19
2,2021-10-01,E02000004,3316.678252,0,0.0,0.0,0,0,0,1,...,0.250803,4.164166,-6227.463237,-0.004362,-0.160598,0.173672,0.180717,-0.165996,-9.386876,135.19
3,2021-10-01,E02000005,2820.681639,0,0.0,0.0,0,0,0,1,...,0.220485,6.134339,436.436763,-0.002964,-0.130190,0.129228,0.144758,-0.196314,-7.416703,135.19
4,2021-10-01,E02000007,2999.628000,0,0.0,0.0,0,0,0,1,...,0.186798,5.266994,-3036.663237,0.011161,-0.150993,0.035123,0.133741,-0.230001,-8.284047,135.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72067,2021-12-11,E02007111,3825.484783,1,71.0,47.0,0,0,0,0,...,0.629448,42.214395,312.236763,-0.020970,0.209927,-0.401807,-0.176251,0.212649,28.663354,146.89
72068,2021-12-11,E02007112,3952.935414,1,71.0,47.0,0,0,0,0,...,0.681750,12.478780,6226.936763,-0.044609,0.236516,-0.360011,-0.162062,0.264951,-1.072262,146.89
72069,2021-12-11,E02007113,4513.575395,1,71.0,47.0,0,0,0,0,...,0.596544,15.832084,1811.836763,-0.032327,0.157646,-0.295962,-0.145625,0.179745,2.281042,146.89
72070,2021-12-11,E02007114,4052.152848,1,71.0,47.0,0,0,0,0,...,0.568014,13.968328,8507.736763,-0.019406,0.111075,-0.274435,-0.138344,0.151215,0.417286,146.89


In [8]:
''' level 1: classic ITS (OLS model) '''
# create formula
level1_vars = ['intervention', 'time1', 'time2'] + [f'dow_{i}' for i in range(1, 7)]
formula = 'count ~ ' + ' + '.join(level1_vars)

# fit the model
md = smf.ols(formula, data)
mdf = md.fit()
print(mdf.summary())

                            OLS Regression Results                            
Dep. Variable:                  count   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     308.3
Date:                Thu, 30 Jan 2025   Prob (F-statistic):               0.00
Time:                        12:10:53   Log-Likelihood:            -5.7083e+05
No. Observations:               72072   AIC:                         1.142e+06
Df Residuals:                   72062   BIC:                         1.142e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     3435.3953     10.268    334.558   

In [9]:
''' level 1: classic ITS (Multilevel model) '''
# Multilevel Model (also known as linear mixed-effect model / multilevel linear regression)
md = smf.mixedlm(formula, data, groups=data[area_key])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)


             Mixed Linear Model Regression Results
Model:              MixedLM   Dependent Variable:   count       
No. Observations:   72072     Method:               REML        
No. Groups:         1001      Scale:                230400.7973 
Min. group size:    72        Log-Likelihood:       -549306.7284
Max. group size:    72        Converged:            Yes         
Mean group size:    72.0                                        
----------------------------------------------------------------
               Coef.    Std.Err.    z    P>|z|  [0.025   0.975] 
----------------------------------------------------------------
Intercept      3435.395   16.370 209.857 0.000 3403.310 3467.480
intervention    106.695    7.756  13.757 0.000   91.494  121.897
time1             5.547    0.449  12.358 0.000    4.667    6.427
time2            -9.837    0.476 -20.674 0.000  -10.769   -8.904
dow_1             8.069    6.787   1.189 0.234   -5.233   21.372
dow_2            -2.514    6.794  -0.37

In [10]:
''' level 2.0: use IMD '''
# # create formula
# level1_vars = ['intervention', 'time1', 'time2'] + [f'dow_{i}' for i in range(1, 7)]
# level2_vars = ['IMD19_centered']
# formula = 'count ~ ' + ' + '.join(level1_vars) + ' + ' + ' + '.join(level2_vars)
# 
# # 对每个level 2变量，加入交互项
# for var in ['intervention', 'time2']:
#     for l2_var in level2_vars:
#         formula += f' + {var}:{l2_var}'
# 
# print('Formula:', formula, '\n')
# 
# # Multilevel Model (also known as linear mixed-effect model / multilevel linear regression)
# md = smf.mixedlm(formula, data, groups=data[area_key])
# mdf = md.fit(method=["lbfgs"])
# print(mdf.summary())
# 
# # Calculate R-squared
# y_true = data['count']
# y_pred = mdf.fittedvalues
# ss_res = np.sum((y_true - y_pred)**2)
# ss_tot = np.sum((y_true - np.mean(y_true))**2)
# r_squared = 1 - (ss_res / ss_tot)
# print(f"R-squared: {r_squared:.4f}")
# 
# # Calculate ICC
# random_effects_var = mdf.cov_re.iloc[0, 0]
# residual_var = mdf.scale
# icc = random_effects_var / (random_effects_var + residual_var)
# print(f"Intraclass Correlation Coefficient (ICC): {icc:.4f}")


' level 2.0: use IMD '

In [11]:
''' level 2.1: use Census '''
# create formula
level1_vars = ['intervention', 'time1', 'time2'] + [f'dow_{i}' for i in range(1, 7)]
level2_vars = [f'{var}_centered' for var in social_vars] + ['fuel_price']
formula = 'count ~ ' + ' + '.join(level1_vars) + ' + ' + ' + '.join(level2_vars)

# 对每个level 2变量，加入交互项
for var in ['intervention', 'time2']:
    for l2_var in level2_vars:
        formula += f' + {var}:{l2_var}'

print('Formula:', formula, '\n')

# Multilevel Model (also known as linear mixed-effect model / multilevel linear regression)
md = smf.mixedlm(formula, data, groups=data[area_key])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# Calculate R-squared
y_true = data['count']
y_pred = mdf.fittedvalues
ss_res = np.sum((y_true - y_pred)**2)
ss_tot = np.sum((y_true - np.mean(y_true))**2)
r_squared = 1 - (ss_res / ss_tot)
print(f"R-squared: {r_squared:.4f}")

# Calculate ICC
random_effects_var = mdf.cov_re.iloc[0, 0]
residual_var = mdf.scale
icc = random_effects_var / (random_effects_var + residual_var)
print(f"Intraclass Correlation Coefficient (ICC): {icc:.4f}")


Formula: count ~ intervention + time1 + time2 + dow_1 + dow_2 + dow_3 + dow_4 + dow_5 + dow_6 + Rden_centered + Sfem_centered + Hedu_centered + Scar_centered + CarWork_centered + Swfh_centered + PTAI_centered + fuel_price + intervention:Rden_centered + intervention:Sfem_centered + intervention:Hedu_centered + intervention:Scar_centered + intervention:CarWork_centered + intervention:Swfh_centered + intervention:PTAI_centered + intervention:fuel_price + time2:Rden_centered + time2:Sfem_centered + time2:Hedu_centered + time2:Scar_centered + time2:CarWork_centered + time2:Swfh_centered + time2:PTAI_centered + time2:fuel_price 



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)


                       Mixed Linear Model Regression Results
Model:                     MixedLM         Dependent Variable:         count       
No. Observations:          72072           Method:                     REML        
No. Groups:                1001            Scale:                      228059.0908 
Min. group size:           72              Log-Likelihood:             -548700.3759
Max. group size:           72              Converged:                  Yes         
Mean group size:           72.0                                                    
-----------------------------------------------------------------------------------
                                Coef.    Std.Err.    z    P>|z|   [0.025    0.975] 
-----------------------------------------------------------------------------------
Intercept                       2601.080  916.585   2.838 0.005   804.607  4397.553
intervention                    7117.136 1005.691   7.077 0.000  5146.018  9088.255
time1          

In [12]:
# 输出用星号标注显著性的系数表，方便加入论文
def add_stars(p_value):
    if p_value < 0.001:
        return "***"
    elif p_value < 0.01:
        return "**"
    elif p_value < 0.05:
        return "*"
    elif p_value < 0.1:
        return "."
    else:
        return ""

results_df = pd.DataFrame({
    'coef': mdf.params,
    'P>|z|': mdf.pvalues,
    '[0.025': mdf.conf_int()[0],
    '0.975]': mdf.conf_int()[1]
})

results_df['sig'] = results_df['P>|z|'].apply(add_stars)  # significance
results_df['coef'] = results_df['coef'].apply(lambda x: f"{x:.3f}") #+ results_df['sig']
results_df['P>|z|'] = results_df['P>|z|'].apply(lambda x: f"{x:.3f}")
results_df['[0.025'] = results_df['[0.025'].apply(lambda x: f"{x:.3f}"[:6])
results_df['0.975]'] = results_df['0.975]'].apply(lambda x: f"{x:.3f}"[:6])

results_df = results_df[[ 'coef','sig', 'P>|z|', '[0.025', '0.975]']]
results_df.to_csv('ITS results/Travel Distance MSOA.csv')  # Travel Distance MSOA, Casual Activity Count MSOA, Commuting Duration MSOA
results_df

,coef,sig,P>|z|,[0.025,0.975]
Intercept,2601.080,**,0.005,804.60,4397.5
intervention,7117.136,***,0.000,5146.0,9088.2
time1,4.213,**,0.005,1.285,7.141
time2,-472.511,***,0.000,-515.8,-429.1
dow_1,10.182,,0.133,-3.114,23.479
dow_2,1.712,,0.804,-11.78,15.206
dow_3,26.455,***,0.000,12.639,40.271
dow_4,-107.281,***,0.000,-121.0,-93.48
dow_5,152.022,***,0.000,137.69,166.35
dow_6,285.563,***,0.000,270.53,300.59
